# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import numpy as np
import patsy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

from patsy import DesignInfo
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [4]:
df.dropna(inplace=True)

df.drop(['Unnamed: 0', 'data_ref', 'id_cliente'], axis='columns', inplace=True, errors='ignore')

df.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

### 1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento)

In [5]:
colunas = [
    'sexo',
    'posse_de_veiculo',
    'posse_de_imovel',
    'qtd_filhos',
    'C(tipo_renda)',
    'C(educacao)',
    'C(estado_civil)',
    'C(tipo_residencia)',
    'idade',
    'tempo_emprego',
    'qt_pessoas_residencia'
]

formula = 'renda ~ ' + ' + '.join(colunas)

y, X = patsy.dmatrices(formula, df)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

### 2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?

In [6]:
def fit_statsmodels(l1_param):
    model = sm.OLS(y_train, X_train)
    
    for alpha in [0, 0.001, 0.005, 0.01, 0.05, 0.1]:
        reg = model.fit_regularized(method='elastic_net', refit=True, L1_wt=l1_param, alpha=alpha)
        predict = reg.predict(X_test)
        R2 = r2_score(y_test, predict)
        
        print(f'R-square {R2} para o alpha = {alpha}')

In [7]:
fit_statsmodels(0.01)

R-square 0.283700027535284 para o alpha = 0
R-square 0.283700027535284 para o alpha = 0.001
R-square 0.283700027535284 para o alpha = 0.005
R-square 0.283700027535284 para o alpha = 0.01
R-square 0.283700027535284 para o alpha = 0.05
R-square 0.28366733614238615 para o alpha = 0.1


> Praticamente todos os modelos regularizados com ridge são iguais frente aos alphas testados.

### 3. Faça o mesmo que no passo 2, com uma regressão LASSO. Qual método chega a um melhor resultado?

In [8]:
fit_statsmodels(1)

R-square 0.283700027535284 para o alpha = 0
R-square 0.283700027535284 para o alpha = 0.001
R-square 0.283700027535284 para o alpha = 0.005
R-square 0.283700027535284 para o alpha = 0.01
R-square 0.283700027535284 para o alpha = 0.05
R-square 0.283700027535284 para o alpha = 0.1


> Todos os modelos são iguais frente aos alphas testados e não apresentam diferença do método ridge

### 4. Rode um modelo *stepwise*. Avalie o $R^2$ na base de testes. Qual o melhor resultado?

**Criando a função stepwise**

In [9]:
def stepwise_selection(X, y, initial_list=[], threshold_in=0.05, threshold_out = 0.05):
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
        if not changed:
            break
    return included

**Criando um DataFrame para X e y que serão usados no stepwise**

In [10]:
step_X = pd.DataFrame(X_train, columns=X.design_info.column_names)
step_y = pd.DataFrame(y_train, columns=['renda'])
step_X.drop(['Intercept'], axis='columns', inplace=True)

**Selecionando as variáveis pelo stepwise**

In [11]:
variaveis = stepwise_selection(step_X, step_y)
print('Variáveis selecionadas:')
print(variaveis)

Variáveis selecionadas:
['tempo_emprego', 'sexo[T.M]', 'C(tipo_renda)[T.Empresário]', 'C(educacao)[T.Superior completo]', 'idade', 'qt_pessoas_residencia']


**Separando os dados de treino e teste com base nas variáveis selecionadas**

In [13]:
step_df = df[['tempo_emprego', 'sexo', 'tipo_renda', 'idade', 'educacao', 'qt_pessoas_residencia', 'renda']]

# Foram definidos como casela de referência os valores selecionados pelo stepwise
colunas = [
    'tempo_emprego',
    'idade',
    'qt_pessoas_residencia',
    "C(sexo, Treatment('F'))",
    "C(tipo_renda, Treatment('Empresário'))",
    "C(educacao, Treatment('Superior completo'))",
]

formula = 'renda ~ ' + ' + '.join(colunas)

y, X = patsy.dmatrices(formula, step_df)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [14]:
fit_statsmodels(1)

R-square 0.2843014514791443 para o alpha = 0
R-square 0.2843014514791443 para o alpha = 0.001
R-square 0.28430155687052194 para o alpha = 0.005
R-square 0.28430155687052194 para o alpha = 0.01
R-square 0.28430155687052194 para o alpha = 0.05
R-square 0.28430155687052194 para o alpha = 0.1


> Praticamente todos os modelos são iguais frente aos alphas.

### 5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?

> A variação entre os modelos testados anteriormente é muito baixa. Praticamente todos são iguais.

### 6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.

**Aplicando a função log na renda**

In [15]:
colunas = [
    'tempo_emprego',
    'idade',
    'qt_pessoas_residencia',
    "C(sexo, Treatment('F'))",
    "C(tipo_renda, Treatment('Empresário'))",
    "C(educacao, Treatment('Superior completo'))",
]

# log na renda
formula = 'np.log(renda) ~ ' + ' + '.join(colunas)

y, X = patsy.dmatrices(formula, step_df)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [16]:
fit_statsmodels(1)

R-square 0.3589946498399448 para o alpha = 0
R-square 0.35920248400378874 para o alpha = 0.001
R-square 0.35777932444418015 para o alpha = 0.005
R-square 0.3583311053207954 para o alpha = 0.01
R-square 0.3512604139845348 para o alpha = 0.05
R-square 0.3512604139845348 para o alpha = 0.1


> Mantendo as caselas de referência definidas anteriormente e transformando a renda em log foi possível obter um R2 de 0.35. Melhor resultado em comparação com os exercícios anteriores.

### 7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [17]:
# Para construir a árvore foram usadas as mesmas colunas selecionadas pelo stepwise
colunas = [
    'tempo_emprego',
    'idade',
    'qt_pessoas_residencia',
    "sexo",
    "tipo_renda",
    "educacao",
    "renda",
]

tree_df = df[colunas]

# tratamento para a coluna sexo: 1 para feminino 0 para masculino
tree_df = tree_df.replace({'sexo': {'F': 1, 'M': 0}})

# dummies
tree_df = pd.get_dummies(tree_df)

# transformando a renda em log
tree_df['renda'] = np.log(tree_df['renda'])

y = tree_df['renda']
X = tree_df.drop(columns=['renda'], axis=1)

# separando base de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

**Criando a árvore de decisão**

In [18]:
reg_tree = DecisionTreeRegressor(random_state=1)
reg_tree.fit(X_train, y_train)

DecisionTreeRegressor(random_state=1)

In [19]:
predict = reg_tree.predict(X_test)
print(f'R-square da árvore de decisão: {r2_score(y_test, predict)}')

R-square da árvore de decisão: 0.3245122823475065


> Ajustando a árvore temos um R2 de 0.32, ligeiramente inferior ao R2 obtido no exercício 6, que foi 0.35.
> Ambos foram ajustados com a base de treino e avaliados com os dados de teste.
> Provavelmente o desempenho foi melhor no statsmodels devido ao ajuste da casela de referência.
